### Disclaimer: This notebook contains functions to load the data from the individual csv tables and further perform data cleaning. It is ment to be executed once. The final dataframe is saved as a .csv file, which can then be worked on with regard to machine learning-based classification 

# Kickstarter Project

### Definition of relevant columns

* backers_count: amount of people pledging money to the project                                     
* category -> 'slug': name of the projects' specific parent- & sub-category (part of json string)
* country: country of the projects creator 
* creator -> 'id': id of the creator -> to be used as categorical variable (part of json string)
* goal: information on the amount of money needed to succeed in the local currency of the project
* launched_at: start date? of the project ()
* deadline: end date of the project ()
* spotlight: project highlighted on the website
* staff_pick: marked by a staff member of kickstarter (more attention drawn towards project)
* state: (successful/failed/canceled/live/suspended) -> exclude 'live' and combine 'canceled', 'suspended' with 'failed'
* static_usd_rate: exchange rate to transform goal in every column from current currency to USD



### Stakeholder: Project creator 
### Question: Is it useful to put much effort into launching a campaign on kickstarter? 
### Measure: Is the campaign likely to succeed or fail?

## Import Libraries

In [164]:
# Libraries

import os, json, re
import pandas as pd 



## Important Functions

In [165]:
######### functions for pre-processing ####################################################################

def extract_year_date_month(df, column):
    '''Takes a column, converts it to datetime, and creates new columns with day, month and year
    The new columns are named:
        - column_weekday
        - column_month
        - column_year
    '''
    
    # Convert column in df to datetime
    df[column] = pd.to_datetime(df[column], unit='s')

    # extract the day, month, and year components
    df[column + '_' + 'weekday'] = df[column].dt.weekday
    df[column + '_' + 'month'] = df[column].dt.month
    #df[column + '_' + 'year'] = df[column].dt.year

    return df


def duration(df, column1, column2):
    '''Returns the duration in days between 2 columns with datetime and puts it into a new colum
        - column1: start date
        - column2: end date
    '''
    df['duration_days'] = (df[column2] - df[column1]).dt.days

    return df

def convert_to_usd(df):
    return round(df['goal'] * df['static_usd_rate'],2)

######### functions for analysing predictions ########################################################## 



## Load data into one dataframe

In [166]:
directory = 'Kickstarter_data/'
data = pd.DataFrame()
relevant_columns = ['backers_count', 'category', 'country', 'creator', 'spotlight', 'staff_pick', 'state', 'static_usd_rate', 'goal', 'launched_at', 'deadline']

for file in sorted(os.listdir(directory)):
    df_temp = pd.read_csv(directory+file)
    data = pd.concat([data, df_temp[relevant_columns]], ignore_index=True)

data.head()

,backers_count,category,country,creator,spotlight,staff_pick,state,static_usd_rate,goal,launched_at,deadline
0,21,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",US,"{""id"":1495925645,""name"":""Daniel"",""is_registere...",True,False,successful,1.000000,200.0,1388011046,1391899046
1,97,"{""id"":54,""name"":""Mixed Media"",""slug"":""art/mixe...",US,"{""id"":1175589980,""name"":""Katherine"",""slug"":""fr...",True,False,successful,1.000000,400.0,1550073611,1551801611
2,88,"{""id"":280,""name"":""Photobooks"",""slug"":""photogra...",US,"{""id"":1196856269,""name"":""MelissaThomas"",""is_re...",True,True,successful,1.000000,27224.0,1478012330,1480607930
3,193,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",IT,"{""id"":1569700626,""name"":""WAO"",""slug"":""wearewao...",True,False,successful,1.136525,40000.0,1540684582,1544309940
4,20,"{""id"":51,""name"":""Software"",""slug"":""technology/...",US,"{""id"":1870845385,""name"":""Kalpit Jain"",""is_regi...",False,False,failed,1.000000,1000.0,1425919017,1428511017


In [167]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   backers_count    209222 non-null  int64  
 1   category         209222 non-null  object 
 2   country          209222 non-null  object 
 3   creator          209222 non-null  object 
 4   spotlight        209222 non-null  bool   
 5   staff_pick       209222 non-null  bool   
 6   state            209222 non-null  object 
 7   static_usd_rate  209222 non-null  float64
 8   goal             209222 non-null  float64
 9   launched_at      209222 non-null  int64  
 10  deadline         209222 non-null  int64  
dtypes: bool(2), float64(2), int64(3), object(4)
memory usage: 14.8+ MB


In [168]:
data = data.drop_duplicates(ignore_index =True)

In [169]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209220 entries, 0 to 209219
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   backers_count    209220 non-null  int64  
 1   category         209220 non-null  object 
 2   country          209220 non-null  object 
 3   creator          209220 non-null  object 
 4   spotlight        209220 non-null  bool   
 5   staff_pick       209220 non-null  bool   
 6   state            209220 non-null  object 
 7   static_usd_rate  209220 non-null  float64
 8   goal             209220 non-null  float64
 9   launched_at      209220 non-null  int64  
 10  deadline         209220 non-null  int64  
dtypes: bool(2), float64(2), int64(3), object(4)
memory usage: 14.8+ MB


## Work on the json string columns

### Extract the 'slug' parameter from the category column and drop the category column

In [170]:
cat_data = data["category"].apply(json.loads)
cat_data = pd.DataFrame(cat_data.tolist())
data['slug'] = cat_data['slug']
data = data.drop("category", axis=1)

### Extract the ID from the creator column and drop the creator column

In [171]:
data["creator_id"] = data["creator"].apply(lambda x: re.findall(r'\d+', x)[0])
data = data.drop("creator", axis=1)


### Exclude rows that have the state "live" 
#### we can't use them because we don't know wether the campaigns will succeed or fail

In [172]:
data = data[data['state'] != 'live'].reset_index(drop=True)

### Assign 1 to state == 'successful' and 0 to 'failed', 'canceled' or 'suspended'

In [173]:
data['state'] = data['state'].apply(lambda x: 1 if x == 'successful' else 0)

data['state'].value_counts()

state
1    117465
0     84446
Name: count, dtype: int64

## Work on the datetime columns

### Convert date-data to type date.time()

In [174]:
data['launched_at'] = pd.to_datetime(data['launched_at'], unit='s')
data['deadline'] = pd.to_datetime(data['deadline'], unit='s')

In [175]:
data["deadline"].sort_values()

62507    2009-05-03 06:59:59
73659    2009-05-16 09:59:00
7917     2009-05-31 11:38:00
9323     2009-06-05 06:59:00
177458   2009-06-06 05:00:00
                 ...        
51475    2019-04-30 13:03:19
168283   2019-05-04 01:25:48
192426   2019-05-08 00:20:48
37779    2019-05-08 22:28:24
139109   2019-05-12 20:17:47
Name: deadline, Length: 201911, dtype: datetime64[ns]

## Work on creator_id column 
### Create a new array, indicating wether a creator had a successful campaign before. 

In [176]:
# data.head()


In [177]:
# creators = data.creator_id.value_counts().to_frame().reset_index()
# multi_creators = creators[creators['count'] > 1]
# multi_creators

### For now: Drop the column

In [178]:
data = data.drop('creator_id', axis =1)

### Extract weekday and month of kickstarter project launch, as well as the duration of the kickstarter project and drop the "launched_at" and "deadline" column

In [179]:
data = extract_year_date_month(data, 'launched_at')
data = duration(data, 'launched_at', 'deadline')

data = data.drop(['launched_at', 'deadline'], axis=1)

### Convert unit of "goal" to USD and drop "static_usd_rate" and "goal" column

In [180]:
data['goal_in_usd'] = data.apply(convert_to_usd, axis=1)
data = data.drop(['static_usd_rate', 'goal'], axis=1)

In [181]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201911 entries, 0 to 201910
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   backers_count        201911 non-null  int64  
 1   country              201911 non-null  object 
 2   spotlight            201911 non-null  bool   
 3   staff_pick           201911 non-null  bool   
 4   state                201911 non-null  int64  
 5   slug                 201911 non-null  object 
 6   launched_at_weekday  201911 non-null  int32  
 7   launched_at_month    201911 non-null  int32  
 8   duration_days        201911 non-null  int64  
 9   goal_in_usd          201911 non-null  float64
dtypes: bool(2), float64(1), int32(2), int64(3), object(2)
memory usage: 11.2+ MB


In [182]:
data.describe().round(2)

,backers_count,state,launched_at_weekday,launched_at_month,duration_days,goal_in_usd
count,201911.00,201911.00,201911.00,201911.00,201911.00,2.019110e+05
mean,146.25,0.58,2.37,6.44,32.33,4.136260e+04
std,878.13,0.49,1.80,3.36,11.72,1.138082e+06
min,0.00,0.00,0.00,1.00,1.00,1.000000e-02
25%,4.00,0.00,1.00,4.00,29.00,1.500000e+03
50%,28.00,1.00,2.00,7.00,30.00,5.000000e+03
75%,90.00,1.00,4.00,9.00,34.00,1.300000e+04
max,105857.00,1.00,6.00,12.00,93.00,1.523501e+08


### Drop duplicates

### Check balance

In [183]:
data.state.value_counts()


state
1    117465
0     84446
Name: count, dtype: int64

#### Classes: 53.18 % succeeded, 46.82 % failed -> Pretty balanced

## Pre-Processing

In [184]:
data.country.value_counts()

country
US    144078
GB     22208
CA      9420
AU      4667
DE      3227
FR      2494
IT      2204
MX      2161
ES      1814
NL      1763
SE      1350
HK       998
NZ       884
DK       876
SG       650
CH       629
IE       611
BE       526
NO       491
AT       472
JP       332
LU        56
Name: count, dtype: int64

### Country to north america True/False

In [185]:
data["north_america"] = data["country"].apply(lambda x: 1 if x in ['US', 'CA'] else 0)

In [186]:
data.north_america.value_counts()

north_america
1    153498
0     48413
Name: count, dtype: int64

In [187]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201911 entries, 0 to 201910
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   backers_count        201911 non-null  int64  
 1   country              201911 non-null  object 
 2   spotlight            201911 non-null  bool   
 3   staff_pick           201911 non-null  bool   
 4   state                201911 non-null  int64  
 5   slug                 201911 non-null  object 
 6   launched_at_weekday  201911 non-null  int32  
 7   launched_at_month    201911 non-null  int32  
 8   duration_days        201911 non-null  int64  
 9   goal_in_usd          201911 non-null  float64
 10  north_america        201911 non-null  int64  
dtypes: bool(2), float64(1), int32(2), int64(4), object(2)
memory usage: 12.7+ MB


In [188]:
data

,backers_count,country,spotlight,staff_pick,state,slug,launched_at_weekday,launched_at_month,duration_days,goal_in_usd,north_america
0,21,US,True,False,1,music/rock,2,12,45,200.0,1
1,97,US,True,False,1,art/mixed media,2,2,20,400.0,1
2,88,US,True,True,1,photography/photobooks,1,11,30,27224.0,1
3,193,IT,True,False,1,fashion/footwear,5,10,41,45461.0,0
4,20,US,False,False,0,technology/software,0,3,30,1000.0,1
...,...,...,...,...,...,...,...,...,...,...,...
201906,57,US,True,False,1,food/drinks,1,2,44,10000.0,1
201907,11,AU,False,False,0,food/cookbooks,5,12,30,5820.6,0
201908,0,US,False,False,0,fashion/childrenswear,4,8,30,8000.0,1
201909,11,US,True,False,1,publishing/children's books,1,2,29,1000.0,1


In [189]:
data = data.drop('country', axis=1)

In [190]:
data

,backers_count,spotlight,staff_pick,state,slug,launched_at_weekday,launched_at_month,duration_days,goal_in_usd,north_america
0,21,True,False,1,music/rock,2,12,45,200.0,1
1,97,True,False,1,art/mixed media,2,2,20,400.0,1
2,88,True,True,1,photography/photobooks,1,11,30,27224.0,1
3,193,True,False,1,fashion/footwear,5,10,41,45461.0,0
4,20,False,False,0,technology/software,0,3,30,1000.0,1
...,...,...,...,...,...,...,...,...,...,...
201906,57,True,False,1,food/drinks,1,2,44,10000.0,1
201907,11,False,False,0,food/cookbooks,5,12,30,5820.6,0
201908,0,False,False,0,fashion/childrenswear,4,8,30,8000.0,1
201909,11,True,False,1,publishing/children's books,1,2,29,1000.0,1


In [191]:
data.slug.value_counts()

slug
design/product design     3711
games/tabletop games      3355
fashion/accessories       3330
comics/comic books        3267
comics/graphic novels     2856
                          ... 
publishing/letterpress      60
games                       54
music/comedy                48
music/chiptune              42
crafts/taxidermy            18
Name: count, Length: 169, dtype: int64

In [192]:
data["slug"] = data["slug"].apply(lambda x: re.split(r'/', x)[0])

In [193]:
data

,backers_count,spotlight,staff_pick,state,slug,launched_at_weekday,launched_at_month,duration_days,goal_in_usd,north_america
0,21,True,False,1,music,2,12,45,200.0,1
1,97,True,False,1,art,2,2,20,400.0,1
2,88,True,True,1,photography,1,11,30,27224.0,1
3,193,True,False,1,fashion,5,10,41,45461.0,0
4,20,False,False,0,technology,0,3,30,1000.0,1
...,...,...,...,...,...,...,...,...,...,...
201906,57,True,False,1,food,1,2,44,10000.0,1
201907,11,False,False,0,food,5,12,30,5820.6,0
201908,0,False,False,0,fashion,4,8,30,8000.0,1
201909,11,True,False,1,publishing,1,2,29,1000.0,1


In [194]:
data.slug.value_counts()

slug
film & video    26925
music           26806
technology      20475
art             20014
publishing      19471
food            15831
games           12818
fashion         11441
comics           8397
design           8127
photography      7996
crafts           7058
theater          6779
journalism       5758
dance            4015
Name: count, dtype: int64

In [195]:
data = pd.get_dummies(data, columns=['slug', 'launched_at_weekday', 'launched_at_month'], drop_first=True)
data

,backers_count,spotlight,staff_pick,state,duration_days,goal_in_usd,north_america,slug_comics,slug_crafts,slug_dance,...,launched_at_month_3,launched_at_month_4,launched_at_month_5,launched_at_month_6,launched_at_month_7,launched_at_month_8,launched_at_month_9,launched_at_month_10,launched_at_month_11,launched_at_month_12
0,21,True,False,1,45,200.0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,97,True,False,1,20,400.0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,88,True,True,1,30,27224.0,1,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,193,True,False,1,41,45461.0,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,20,False,False,0,30,1000.0,1,False,False,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201906,57,True,False,1,44,10000.0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
201907,11,False,False,0,30,5820.6,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
201908,0,False,False,0,30,8000.0,1,False,False,False,...,False,False,False,False,False,True,False,False,False,False
201909,11,True,False,1,29,1000.0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [196]:
data.to_csv('cleaned_data.csv', index=False)